In [2]:
import os
os.chdir('../')
%pwd

'c:\\Users\\Omar Hady\\Data Projects\\Machine Learning\\End-To-End Cancer Evaluation\\End-To-End-Cancer-Classification-Using-MLflow'

In [3]:
from src.Chest_Cancer_Classification.constants import *
from src.Chest_Cancer_Classification.utils.common import create_directories, read_yaml
import tensorflow as  tf

In [17]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentated: bool
    params_image_size: float


In [15]:
from src.Chest_Cancer_Classification.constants import *
from src.Chest_Cancer_Classification.utils.common import create_directories, read_yaml
from src.Chest_Cancer_Classification.entity.config_entity import DataIngestionConfiguration, BaseModelConfig


class ConfigurationManager:
    '''
    General Configuration class containing configuration methods for every component
    '''
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion(self)-> DataIngestionConfiguration:

        '''
        Method for adding configurations to data ingesting component
        '''
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfiguration(
            root_dir=config.root_dir,
            source_url= config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config
    

    def get_base_model(self)-> BaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        base_model_config = BaseModelConfig(
            root_dir = config.root_dir,
            base_model_path = config.base_model_path,
            updated_base_model_path = config.updated_base_model_path,
            params_image_size = self.params.IMAGE_SIZE,
            params_learning_rate= self.params.LEARNING_RATE,
            params_include_top= self.params.INCLUDE_TOP,
            params_weight= self.params.WEIGHTS,
            params_classes= self.params.CLASSES)
        return base_model_config
    
    def training_model_config(self)->TrainingConfig:
        training= self.config.training
        base_model = self.config.prepare_base_model
        params = self.params
        data = os.path.join(self.config.data_ingestion.unzip_dir, 'Chest-CT-Scan-data')

        training_config = TrainingConfig(

            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_model_path=Path(base_model.updated_base_model_path),
            training_data=Path(data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentated=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE)
        return training_config

In [9]:
class Training:
    def __init__(self, config:TrainingConfig):
        self.config =  config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_model_path)

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
        
        def train(self):
            self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
            self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

            self.model.fit(
                self.train_generator,
                epochs=self.config.params_epochs,
                steps_per_epoch=self.steps_per_epoch,
                validation_steps=self.validation_steps,
                validation_data=self.valid_generator
            )

            self.save_model(
                path=self.config.trained_model_path,
                model=self.model
            )

        @staticmethod
        def save_model(path:Path, model: tf.keras.Model):
            model.save(path)


In [18]:
#Pipeline
configuration = ConfigurationManager()
training_config = configuration.training_model_config()
training = Training(config = training_config)
training.get_base_model()
training.train_valid_generator()
training.train()

[2024-11-19 18:26:13,649 - INFO - common - config\config.yaml loaded successfully]
[2024-11-19 18:26:13,652 - INFO - common - params.yaml loaded successfully]
[2024-11-19 18:26:13,653 - INFO - common - created directory at: artifacts]
[2024-11-19 18:26:13,967 - WARNING - saving_utils - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'artifacts\\data_ingestion\\Chest-CT-Scan-data'